In [1]:
# import os
# os.environ["R_HOME"] = "/home/ld/anaconda3/envs/r/lib/R"
import sys
sys.path.append('../')
import scDesign3

The R project used is located at D:/Code_env/R-4.3.1


In [2]:
import anndata as ad
example_sce = ad.read_h5ad('test_data/VISIUM.h5ad')
example_sce.obs['test'] = ['test' for i in range(example_sce.n_obs)]
example_sce = example_sce[0:10,0:10]

In [3]:
example_sce

View of AnnData object with n_obs × n_vars = 10 × 10
    obs: 'nCount_Spatial', 'nFeature_Spatial', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'spatial1', 'spatial2', 'cell_type', 'test'
    var: 'name'

construct_data interface

In [ ]:
## windows
bpparam = scDesign3.get_bpparam("SnowParam",stop_on_error = False)

## linux/mac
# bpparam = test.get_bpparam("MulticoreParam",stop_on_error = False)

In [4]:
# test = scDesign3.scDesign3(n_cores=3,parallelization="bpmapply",bpparam=bpparam)
test = scDesign3.scDesign3(n_cores=1,parallelization="mcmapply")

In [5]:
const_data = test.construct_data(
    anndata=example_sce,
    corr_formula="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
)

In [ ]:
const_data['dat']

In [ ]:
print(test.construct_data_res)

In [ ]:
# SummarizedExperiment = packages.importr('SummarizedExperiment')
# print(SummarizedExperiment.assay(test._sce,test._assay_use))
# print(SummarizedExperiment.colData(test._sce))
# print(SummarizedExperiment.rowData(test._sce))

fit_marginal interface

In [ ]:
marginal = test.fit_marginal(
    # data=const_data,
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
    trace=True,
)

In [ ]:
print(test.fit_marginal_res)

fit_copula interface

In [ ]:
copula = test.fit_copula(
    # family_use="nb",
    n_cores=1,
    # copula = "vine",
    parallelization="mcmapply",
    # parallelization="bpmapply",
    # bpparam=bpparam,
    important_feature="auto",
)

In [ ]:
print(copula)

extract paras interface

In [ ]:
paras = test.extract_para()

In [ ]:
print(paras)

simu_new interface


In [ ]:
new = test.simu_new()

In [ ]:
print(new)

scdesign3 interface

In [ ]:
whole_test = scDesign3.scDesign3(n_cores=1)
whole = whole_test.scdesign3(    
    anndata=example_sce,
    corr_formula="1",
    default_assay_name="counts",
    celltype='cell_type',
    spatial=['spatial1','spatial2'],
    mu_formula="s(spatial1,spatial2,bs='gp',k=2)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
    trace=True,
    important_feature="auto",
    # parallelization="mcmapply",
    # n_cores=1,
    )

In [ ]:
print(whole_test.whole_pipeline_res)

In [ ]:
whole["marginal_list"]

perform_lrt interface

In [9]:
test1 = scDesign3.scDesign3(n_cores=1,parallelization="mcmapply")
test2 = scDesign3.scDesign3(n_cores=1,parallelization="mcmapply")

In [16]:
my_data = test1.construct_data(
anndata = example_sce,
default_assay_name = "counts",
celltype = "cell_type",
spatial= ["spatial1","spatial2"],
corr_formula = "cell_type"
)
my_data2 = test2.construct_data(
anndata = example_sce,
default_assay_name = "counts",
celltype = "cell_type",
spatial= ["spatial1","spatial2"],
corr_formula = "1",
ncell = 10
)

In [17]:
test1.fit_marginal(
    # data=const_data,
    mu_formula="1",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
)
marginal1 = test1.fit_marginal_res

test2.fit_marginal(
    # data=const_data,
    mu_formula="s(spatial1,spatial2,bs='gp',k=3)",
    sigma_formula="1",
    family_use="nb",
    usebam=False,
)
marginal2 = test2.fit_marginal_res

In [21]:
marginal1 = {key: value.rx2("fit") for key,value in marginal1.items()}
marginal2 = {key: value.rx2("fit") for key,value in marginal2.items()}

In [22]:
scDesign3.perform_lrt(marginal2,marginal1)

,same_model,LogLik_alter,LogLik_null,df_alter,df_null,p_value
Calb2,1.0,-26.541848,-33.801188,3.000179,1.0,0.000704
Gng4,1.0,-25.430800,-29.461979,3.000646,1.0,0.017766
Ttr,1.0,-14.533833,-15.700972,3.000007,1.0,0.311258
S100a5,1.0,-12.476353,-19.743043,4.081265,1.0,0.002459
Nrgn,1.0,-46.844004,-49.373142,4.097314,1.0,0.177682
Doc2g,1.0,-20.517832,-25.194593,3.001276,1.0,0.009323
Ppp1r1b,1.0,-25.195347,-29.929347,4.123708,1.0,0.026270
Pcbp3,1.0,-25.598560,-30.186020,3.000326,1.0,0.010182
Fabp7,1.0,-21.232512,-34.053669,3.001523,1.0,0.000003
Gpsm1,1.0,-28.188423,-31.319739,3.000191,1.0,0.043668


Conversion test

In [ ]:
from rpy2.rlike.container import OrdDict
from rpy2.robjects import NULL, ListVector, StrVector, conversion, default_converter, numpy2ri, pandas2ri, r, packages

@default_converter.py2rpy.register(OrdDict)
def convert_ord_dict(ord_dict):
    with (default_converter + pandas2ri.converter + numpy2ri.converter).context():
        ord_dict = {k: conversion.get_conversion().py2rpy(v) for k, v in ord_dict.items()}
    return ListVector(ord_dict)


convert = numpy2ri.converter + default_converter + pandas2ri.converter

In [ ]:
with convert.context():
    a = test.fit_marginal_res.rx2("Calb2")
    b = r("mgcv::gam(Sepal.Length~Sepal.Width,data=iris)")

print(b)

In [ ]:
a["fit"]

In [ ]:
print(test.fit_marginal_res.rx2("Calb2"))